In [39]:
import pandas as pd
import xmltodict
import requests
import sys
sys.path.append('..')

from utils.database import push_dataframe_to_postgres
from auth.headway_auth import CHARLES_HEADWAY_AUTH

doc = xmltodict.parse(
    requests.get('http://webservices.nextbus.com/service/publicXMLFeed?command=routeList&a=sf-muni').text)

In [40]:
route_df = pd.DataFrame.from_dict(
    doc['body']['route']).rename(columns={'@tag': 'route_id', '@title': 'route_name'})

In [41]:
to_concat = []
for id in route_df['route_id']:
    route_doc = xmltodict.parse(
        requests.get('http://webservices.nextbus.com/service/publicXMLFeed?command=routeConfig&a=sf-muni&r=%s' % id).text)
    this_route = (pd.DataFrame.from_dict(route_doc['body']['route']['stop']).rename(columns={
                '@lat': 'lat',
                '@lon': 'lon',
                '@stopId': 'stop_id',
                '@title': 'stop_name',
                '@tag': 'xml_tag'
            }))
    this_route['route_id'] = id
    to_concat.append(this_route)

In [42]:
stop_df = pd.concat(to_concat)[['route_id', 'stop_id', 'stop_name', 'xml_tag', 'lat', 'lon']]

In [43]:
push_dataframe_to_postgres(route_df, CHARLES_HEADWAY_AUTH, 'routes')
push_dataframe_to_postgres(stop_df, CHARLES_HEADWAY_AUTH, 'stops')